***Stock Market Prediction using Machine Learning***

In [6]:
!cd

C:\Users\Admin\Downloads\MyStreamlitApp


In [4]:
pip install vaderSentiment


Note: you may need to restart the kernel to use updated packages.


In [15]:
from alpha_vantage.timeseries import TimeSeries
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
import pandas as pd

# Replace with your Alpha Vantage API key (remove before sharing)
api_key = 'SYE7E25EB4O6HTL9'  # Consider environment variables for security
stock_symbol = 'MSFT'

# Fetch stock data
ts = TimeSeries(key=api_key, output_format='pandas')
data, _ = ts.get_daily(symbol=stock_symbol, outputsize='full')
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)

# Technical indicators (MUST MATCH APP)
data['MA_50'] = data['Close'].rolling(50).mean()
data['MA_200'] = data['Close'].rolling(200).mean()
data.dropna(inplace=True)

# Create and save scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume', 'MA_50', 'MA_200']])
joblib.dump(scaler, 'stock_scaler.pkl')  # Save for Streamlit app

# Create sequences
n_days = 60
X, y = [], []
for i in range(n_days, len(scaled_data)):
    X.append(scaled_data[i-n_days:i, :])
    y.append(scaled_data[i, 3])  # Close price index
X, y = np.array(X), np.array(y)

# LSTM Model
lstm_model = Sequential([
    Input(shape=(n_days, X.shape[2])),
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X, y, epochs=100, batch_size=32)  # Fixed X_lstm to X
lstm_model.save('lstm_model.keras')

# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=300)
rf_model.fit(X.reshape(X.shape[0], -1), data['Close'][n_days:])
joblib.dump(rf_model, 'rf_model.pkl')

print("All models and scaler saved!")

Epoch 1/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - loss: 0.0058
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - loss: 7.2462e-04
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 6.2139e-04
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 7.0447e-04
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 5.7559e-04
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 7.3093e-04
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 4.2768e-04
Epoch 8/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 4.7619e-04
Epoch 9/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 4.0883e-04
Epoch 10/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 4.5488e-04
Epoch 11/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 6.3034e-04
Epoch 12/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 3.9676e-04
Epoch 13/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 4.0344e-04
Epoch 14/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/

In [ ]:
!streamlit run App.py
